In [2]:
#imports all needed modules

import pyart
import numpy as np
from datetime import timedelta
import glob
from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import cartopy
import gc
import pandas as pd


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\julia\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\julia\Desktop\Research\radarcalc.py:81: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


## Get Dealised Stationary Ka VAD files (not bearing corrected)

In [3]:
#SMALL_SIZE = 30
#MEDIUM_SIZE = 40
#BIGGER_SIZE = 60

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
#plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
#plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


#CHANGE directories to where you have the product raw and GPS files
# old directories for pc
#vad_files = sorted(glob.glob('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\product_raw_ka2\\*'))
#df = pd.read_csv('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\GPS_Ka2_20220523.txt', dtype=str)

vad_files = sorted(glob.glob('C:\\Users\\julia\\Desktop\\Research\\product_raw_ka1\\*'))

df = pd.read_csv('C:\\Users\\julia\\Desktop\\Research\\GPS_Ka1_20220523.txt', dtype=str)

# ask how he knows what indeces the files belong to, i.e. thefile[-14:8]

#Loops over each rdar file to get dealias ka data
for thefile in vad_files:
    
    if (thefile[-14:-8] in df['hhmmss[UTC]'].values) != True:   #makes sure the times are the same from both files
        continue
    else:
        radar = pyart.io.read(thefile)
    
        df['ddmmyy']+df['hhmmss[UTC]'] 
        df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']] #adds date column to time column
        beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amin(np.unique(radar.time['data'])) #finds index where VAD starts
        endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amax(np.unique(radar.time['data'])) #finds index where VAD stops
        dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)] #uses prior beginning and end indeces to find all times for VAD
        dfscan = dfscan.astype({'Bearing[degrees]': 'float'}) #looking at Bearing column and changing all entries to floats so they can be decimals
        #     if np.amax(dfscan['Bearing[degrees]'])-np.amin(dfscan['Bearing[degrees]']) > 180:
        #         dfscan['Bearing[degrees]'][dfscan['Bearing[degrees]']>180] -=360
        
        #Find where VAD where nearly stationary (<5 knotss)
        if np.logical_and(np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0]>20),dfscan['Velocity[knots]'].astype(float).mean()<5.0):#,dfscan['Bearing[degrees]'].astype(float).var()<100.0)):
            print(thefile[:-8])   #above line checks to make sure the scan type is a ppi, that the angle is greater than 20, and that velocity is less than 5 m/s-- these are the most desirable conditions for the data
            
            radar = dealias_Ka(radar)
            
            pyart.io.write_cfradial(thefile[:-8]+".nc", radar) #creates netcdf file with dealiased data so we dont have to dealias it every time, which is the goal of this code
#             #radar.fields['velocity']['data']

#             print('velocity [kts]',dfscan['Velocity[knots]'].astype(float).mean(),'+-',dfscan['Velocity[knots]'].astype(float).var())
#             speed = dfscan['Velocity[knots]'].astype(float).mean()
#             print('bearing',dfscan['Bearing[degrees]'].astype(float).mean(),'+-',dfscan['Bearing[degrees]'].astype(float).var())
#             bearing = dfscan['Bearing[degrees]'].astype(float).mean()
#             print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
#             lat = dfscan['Latitude'].astype(float).mean()
#             print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
#             lon = dfscan['Longitude'].astype(float).mean()

#             VAD = pyart.retrieve.vad_browning(radar, 'corrected_velocity', z_want=np.arange(50,5000,10),gatefilter=None)
#             vadu = VAD.u_wind*1.94384
#             vadv = VAD.v_wind*1.94384
#             vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), bearing*units('degrees'))
#             vadu = (vadu)
#             vadv = (vadv)

#             fig = plt.figure(figsize=(50, 40),facecolor='white')
#             display = pyart.graph.RadarDisplay(radar)
#             ax3 = fig.add_subplot(2,2,3)

#             hodo = metpy.plots.Hodograph(ax3,component_range=100.)
#             hodo.add_grid(increment=5)

#             hodo.plot(vadu,vadv)
#             ax3.set_xlabel('U-component Wind [knots]')
#             ax3.set_ylabel('V-component Wind [knots]')
#             ax3.set_title('Uncorrected VAD')



#             ax1 = fig.add_subplot(2,2,1)
#             display.plot_ppi('corrected_velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
#                                  colorbar_label=r'Radial Velocity [m s$^{-1}$]', title='Uncorrected Radial Velocity')
#             print(radar.azimuth['data'])
#             (radar.azimuth['data'])+=(bearing*(-1)+90)%360
#         #         radar.azimuth['data'] = np.where((radar.azimuth['data']>90)&(radar.azimuth['data']<360),
#         #                  (360-radar.azimuth['data']+90),(radar.azimuth['data']*(-1)+90))
#             #(radar.azimuth['data'])+=(bearing*(-1)+90)

#         #         ax3 = fig.add_subplot(2,2,1)
#         #         display.plot_ppi('corrected_velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
#         #                              colorbar_label=r'Radial Velocity [m s$^{-1}$]', title='Corrected Radial Velocity')

#             VAD = pyart.retrieve.vad_browning(radar, 'corrected_velocity', z_want=np.arange(50,5001,10),gatefilter=None)
#             vadu = VAD.u_wind*1.94384
#             vadv = VAD.v_wind*1.94384
#             vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), (bearing*(-1)+90)*units('degrees'))
#             vadu = (vadu)#-vehicleu)
#             vadv = (vadv)#-vehiclev)#/0.514444

#             ax4 = fig.add_subplot(2,2,4)

#             hodo = metpy.plots.Hodograph(ax4,component_range=100.)
#             hodo.add_grid(increment=5)

#             hodo.plot(vadu,vadv)
#             ax4.set_xlabel('U-component Wind [knots]')
#             ax4.set_ylabel('V-component Wind [knots]')
#             ax4.set_title('Bearing corrected VAD')

#             for i,z in enumerate([0,95,195,295,395,495]):
#                 if np.isnan(vadu[z]):
#                     pass
#                 else:
#                     circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
#                     ax4.add_patch(circle)
#                     label = ax4.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")



#             r=35
#             radar_namelist, radar_list = get_radar_from_aws(
#                 'KUDX', datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S'), 
#                 datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S')+timedelta(minutes=5))
#             scantime88d = datetime.strptime(str(radar_namelist)[46:61], '%Y%m%d_%H%M%S')
#             swp_id = 0
#             radar = radar_list[0].extract_sweeps([0])
#             time_text = ' ' + str(radar.time['units'][-9:1])
#             title ='KUDX PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
#             display = pyart.graph.RadarMapDisplay(radar)
#             ax2 = fig.add_subplot(2,2,2,projection=display.grid_projection)
#             display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
#                                  colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
#                                  min_lon=getLocation(lat, lon, 270, r)[1],
#                                  max_lon=getLocation(lat, lon, 90, r)[1],
#                                  min_lat=getLocation(lat, lon, 180, r)[0],
#                                  max_lat=getLocation(lat, lon, 0, r)[0],
#                                  embellish=False)
#             ax2.plot(lon,lat,'k*',ms=50,transform=ccrs.PlateCarree())
#             ax2.plot(-104.4056,45.0223,'b*',ms=50,transform=ccrs.PlateCarree())
#             #ax2.plot(-103.7701,44.59018,'b*',ms=50,transform=ccrs.PlateCarree())
#             ax2.set_title(title)

#             plt.suptitle(thefile.split('\\')[-1].split('.')[0][3:])#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
#             fname = thefile.split('\\')[-1].split('.')[0][3:]
# #         fig.savefig(f'/Users/joshu/Desktop/{fname}nondealise.png',dpi=300)
#         plt.close('all')
#         fig.clf()
#         gc.collect()

C:\Users\julia\anaconda3\Lib\site-packages\pyart\io\sigmet.py:146: UserWarning: Unknown type: 17, returning raw data
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\Users\julia\anaconda3\Lib\site-packages\pyart\io\sigmet.py:146: RuntimeWarning: invalid value encountered in sqrt
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\Users\julia\anaconda3\Lib\site-packages\pyart\io\sigmet.py:146: UserWarning: Unknown type: 17, returning raw data
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\Users\julia\anaconda3\Lib\site-packages\pyart\io\sigmet.py:146: RuntimeWarning: invalid value encountered in sqrt
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\Users\julia\anaconda3\Lib\site-packages\pyart\io\sigmet.py:146: UserWarning: Unknown type: 17, returning raw data
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\Users\julia\anaconda3\Lib\site-packag

C:\Users\julia\Desktop\Research\product_raw_ka1\Ka1220523205632


C:\Users\julia\anaconda3\Lib\site-packages\pyart\util\sigmath.py:63: RuntimeWarning: invalid value encountered in sqrt
  std_dev = np.sqrt(-2 * np.log(norm)) * (half_width) / np.pi


KeyboardInterrupt: 

## Random Testing Below

In [ ]:
for thefile in vad_files[181:182]:

    radar = pyart.io.read(thefile)
    print(thefile,radar.fixed_angle['data'])

    df = pd.read_csv('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\GPS_Ka1_20220612.txt', dtype=str)
    
    
    df['ddmmyy']+df['hhmmss[UTC]']
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amin(np.unique(radar.time['data']))
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amax(np.unique(radar.time['data']))
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    if np.amax(dfscan['Bearing[degrees]'])-np.amin(dfscan['Bearing[degrees]']) > 180:
        dfscan['Bearing[degrees]'][dfscan['Bearing[degrees]']>180] -=360

    if np.logical_and(np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0]>20),np.logical_and(dfscan['Velocity[knots]'].astype(float).var()<50.0,dfscan['Bearing[degrees]'].astype(float).var()<100.0)):
        #radar = dealias_Ka(radar)


        print('velocity [kts]',dfscan['Velocity[knots]'].astype(float).mean(),'+-',dfscan['Velocity[knots]'].astype(float).var())
        speed = dfscan['Velocity[knots]'].astype(float).mean()
        print('bearing',dfscan['Bearing[degrees]'].astype(float).mean(),'+-',dfscan['Bearing[degrees]'].astype(float).var())
        bearing = dfscan['Bearing[degrees]'].astype(float).mean()
        print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
        lat = dfscan['Latitude'].astype(float).mean()
        print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
        lon = dfscan['Longitude'].astype(float).mean()

        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5000,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), bearing*units('degrees'))
        vadu = (vadu)
        vadv = (vadv)

        fig = plt.figure(figsize=(50, 40),facecolor='white')
        display = pyart.graph.RadarDisplay(radar)
        ax3 = fig.add_subplot(2,2,3)

        hodo = metpy.plots.Hodograph(ax3,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax3.set_xlabel('U-component Wind [knots]')
        ax3.set_ylabel('V-component Wind [knots]')
        ax3.set_title('Uncorrected VAD')



        
        
        

        ax1 = fig.add_subplot(2,2,1)
        display.plot_ppi('velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
                             colorbar_label=r'Radial Velocity [m s$^{-1}$]', title='Uncorrected Radial Velocity')
        print(radar.azimuth['data'])
#         (radar.azimuth['data'])+=(360-bearing+90)
        (radar.azimuth['data'])+=(bearing*(-1)+90)%360
        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5001,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), ((bearing+180)%360)*units('degrees'))
        vadu = (vadu)#+vehicleu.magnitude)
        vadv = (vadv)#+vehiclev.magnitude)#/0.514444
        
        ax4 = fig.add_subplot(2,2,4)

        hodo = metpy.plots.Hodograph(ax4,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax4.set_xlabel('U-component Wind [knots]')
        ax4.set_ylabel('V-component Wind [knots]')
        ax4.set_title('Bearing corrected VAD')

        for i,z in enumerate([0,95,195,295,395,495]):
            if np.isnan(vadu[z]):
                pass
            else:
                circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
                ax4.add_patch(circle)
                label = ax4.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")



        r=35
        radar_namelist, radar_list = get_radar_from_aws(
            'KUDX', datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S'), 
            datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S')+timedelta(minutes=5))
        scantime88d = datetime.strptime(str(radar_namelist)[46:61], '%Y%m%d_%H%M%S')
        swp_id = 0
        radar = radar_list[0].extract_sweeps([0])
        time_text = ' ' + str(radar.time['units'][-9:1])
        title ='KUDX PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
        display = pyart.graph.RadarMapDisplay(radar)
        ax2 = fig.add_subplot(2,2,2,projection=display.grid_projection)
        display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
                             colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
                             min_lon=getLocation(lat, lon, 270, r)[1],
                             max_lon=getLocation(lat, lon, 90, r)[1],
                             min_lat=getLocation(lat, lon, 180, r)[0],
                             max_lat=getLocation(lat, lon, 0, r)[0],
                             embellish=False)
        ax2.plot(lon,lat,'k*',ms=50,transform=ccrs.PlateCarree())
        ax2.plot(-104.4056,45.0223,'b*',ms=50,transform=ccrs.PlateCarree())
        #ax2.plot(-103.7701,44.59018,'b*',ms=50,transform=ccrs.PlateCarree())
        ax2.set_title(title)

        plt.suptitle(thefile.split('\\')[-1].split('.')[0][3:])#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
        fname = thefile.split('\\')[-1].split('.')[0][3:]
        fig.savefig(f'/Users/julia/Desktop/{fname}nondealise.png',dpi=300)
        plt.close('all')
        fig.clf()
        gc.collect()

In [ ]:
for thefile in vad_files[183:184]:

    radar = pyart.io.read(thefile)
    print(thefile,radar.fixed_angle['data'])

    df = pd.read_csv('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\GPS_Ka1_20220612.txt', dtype=str)
    
    
    df['ddmmyy']+df['hhmmss[UTC]']
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amin(np.unique(radar.time['data']))
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amax(np.unique(radar.time['data']))
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    if np.amax(dfscan['Bearing[degrees]'])-np.amin(dfscan['Bearing[degrees]']) > 180:
        dfscan['Bearing[degrees]'][dfscan['Bearing[degrees]']>180] -=360

    if np.logical_and(np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0]>20),np.logical_and(dfscan['Velocity[knots]'].astype(float).var()<50.0,dfscan['Bearing[degrees]'].astype(float).var()<100.0)):
        #radar = dealias_Ka(radar)



        print('velocity [kts]',dfscan['Velocity[knots]'].astype(float).mean(),'+-',dfscan['Velocity[knots]'].astype(float).var())
        speed = dfscan['Velocity[knots]'].astype(float).mean()
        print('bearing',dfscan['Bearing[degrees]'].astype(float).mean(),'+-',dfscan['Bearing[degrees]'].astype(float).var())
        bearing = dfscan['Bearing[degrees]'].astype(float).mean()
        print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
        lat = dfscan['Latitude'].astype(float).mean()
        print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
        lon = dfscan['Longitude'].astype(float).mean()

        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5000,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), bearing*units('degrees'))
        vadu = (vadu)
        vadv = (vadv)

        fig = plt.figure(figsize=(50, 40),facecolor='white')
        display = pyart.graph.RadarDisplay(radar)
        ax3 = fig.add_subplot(2,2,3)

        hodo = metpy.plots.Hodograph(ax3,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax3.set_xlabel('U-component Wind [knots]')
        ax3.set_ylabel('V-component Wind [knots]')
        ax3.set_title('Uncorrected VAD')



        
        
        

        ax1 = fig.add_subplot(2,2,1)
        display.plot_ppi('velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
                             colorbar_label=r'Radial Velocity [m s$^{-1}$]', title='Uncorrected Radial Velocity')
        print(radar.azimuth['data'])
#         (radar.azimuth['data'])+=(360-bearing+90)
        (radar.azimuth['data'])+=(bearing*(-1)-90)%360
        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5001,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), ((bearing+180)%360)*units('degrees'))
        vadu = (vadu)#+vehicleu.magnitude)
        vadv = (vadv)#+vehiclev.magnitude)#/0.514444
        
        ax4 = fig.add_subplot(2,2,4)

        hodo = metpy.plots.Hodograph(ax4,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax4.set_xlabel('U-component Wind [knots]')
        ax4.set_ylabel('V-component Wind [knots]')
        ax4.set_title('Bearing corrected VAD')

        for i,z in enumerate([0,95,195,295,395,495]):
            if np.isnan(vadu[z]):
                pass
            else:
                circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
                ax4.add_patch(circle)
                label = ax4.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")



        r=35
        radar_namelist, radar_list = get_radar_from_aws(
            'KUDX', datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S'), 
            datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S')+timedelta(minutes=5))
        scantime88d = datetime.strptime(str(radar_namelist)[46:61], '%Y%m%d_%H%M%S')
        swp_id = 0
        radar = radar_list[0].extract_sweeps([0])
        time_text = ' ' + str(radar.time['units'][-9:1])
        title ='KUDX PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
        display = pyart.graph.RadarMapDisplay(radar)
        ax2 = fig.add_subplot(2,2,2,projection=display.grid_projection)
        display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
                             colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
                             min_lon=getLocation(lat, lon, 270, r)[1],
                             max_lon=getLocation(lat, lon, 90, r)[1],
                             min_lat=getLocation(lat, lon, 180, r)[0],
                             max_lat=getLocation(lat, lon, 0, r)[0],
                             embellish=False)
        ax2.plot(lon,lat,'k*',ms=50,transform=ccrs.PlateCarree())
        ax2.plot(-104.4056,45.0223,'b*',ms=50,transform=ccrs.PlateCarree())
        #ax2.plot(-103.7701,44.59018,'b*',ms=50,transform=ccrs.PlateCarree())
        ax2.set_title(title)

        plt.suptitle(thefile.split('\\')[-1].split('.')[0][3:])#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
        fname = thefile.split('\\')[-1].split('.')[0][3:]
        fig.savefig(f'/Users/joshu/Desktop/{fname}nondealise.png',dpi=300)
        plt.close('all')
        fig.clf()
        gc.collect()

In [ ]:
for thefile in vad_files[195:196]:
#for thefile in vad_files[150:151]:
    
    radar = pyart.io.read(thefile)
    print(thefile,radar.fixed_angle['data'])

    df = pd.read_csv('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\GPS_Ka1_20220612.txt', dtype=str)
    
    
    df['ddmmyy']+df['hhmmss[UTC]']
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amin(np.unique(radar.time['data']))
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amax(np.unique(radar.time['data']))
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    if np.amax(dfscan['Bearing[degrees]'])-np.amin(dfscan['Bearing[degrees]']) > 180:
        dfscan['Bearing[degrees]'][dfscan['Bearing[degrees]']>180] -=360

    if np.logical_and(np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0]>20),np.logical_and(dfscan['Velocity[knots]'].astype(float).var()<50.0,dfscan['Bearing[degrees]'].astype(float).var()<100.0)):
        #radar = dealias_Ka(radar)



        print('velocity [kts]',dfscan['Velocity[knots]'].astype(float).mean(),'+-',dfscan['Velocity[knots]'].astype(float).var())
        speed = dfscan['Velocity[knots]'].astype(float).mean()
        print('bearing',dfscan['Bearing[degrees]'].astype(float).mean(),'+-',dfscan['Bearing[degrees]'].astype(float).var())
        bearing = dfscan['Bearing[degrees]'].astype(float).mean()
        print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
        lat = dfscan['Latitude'].astype(float).mean()
        print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
        lon = dfscan['Longitude'].astype(float).mean()

        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5000,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), bearing*units('degrees'))
        vadu = (vadu)
        vadv = (vadv)

        fig = plt.figure(figsize=(50, 40),facecolor='white')
        display = pyart.graph.RadarDisplay(radar)
        ax3 = fig.add_subplot(2,2,3)

        hodo = metpy.plots.Hodograph(ax3,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax3.set_xlabel('U-component Wind [knots]')
        ax3.set_ylabel('V-component Wind [knots]')
        ax3.set_title('Uncorrected VAD')
        


        ax1 = fig.add_subplot(2,2,1)
        display.plot_ppi('velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
                             colorbar_label=r'Radial Velocity [m s$^{-1}$]', title='Uncorrected Radial Velocity')
        bearing=182.53#129.86
        (radar.azimuth['data'])+=(bearing*(-1)-90)%360
        print(radar.azimuth['data'])
        
#         radar.azimuth['data'] = np.where((radar.azimuth['data']>90)&(radar.azimuth['data']<360),
#                  ((360-radar.azimuth['data']+90)+180)%360,((radar.azimuth['data']*(-1)+90)+180)%360)
        
#         print(radar.azimuth['data'])
        
        VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5001,10),gatefilter=None)
        vadu = VAD.u_wind*1.94384
        vadv = VAD.v_wind*1.94384
        vehicleu,vehiclev = mpcalc.wind_components(speed*units('kts'), ((180-bearing+90) %360)*units('degrees'))
        vadu = (vadu)#-vehicleu)
        vadv = (vadv)#-vehiclev)#/0.514444
        
        ax4 = fig.add_subplot(2,2,4)

        hodo = metpy.plots.Hodograph(ax4,component_range=100.)
        hodo.add_grid(increment=5)

        hodo.plot(vadu,vadv)
        ax4.set_xlabel('U-component Wind [knots]')
        ax4.set_ylabel('V-component Wind [knots]')
        ax4.set_title('Bearing corrected VAD')

        for i,z in enumerate([0,95,195,295,395,495]):
            if np.isnan(vadu[z]):
                pass
            else:
                circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
                ax4.add_patch(circle)
                label = ax4.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")



        r=35
        radar_namelist, radar_list = get_radar_from_aws(
            'KUDX', datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S'), 
            datetime.strptime(thefile.split('\\')[-1].split('.')[0][3:],'%y%m%d%H%M%S')+timedelta(minutes=5))
        scantime88d = datetime.strptime(str(radar_namelist)[46:61], '%Y%m%d_%H%M%S')
        swp_id = 0
        radar = radar_list[0].extract_sweeps([0])
        time_text = ' ' + str(radar.time['units'][-9:1])
        title ='KUDX PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
        display = pyart.graph.RadarMapDisplay(radar)
        ax2 = fig.add_subplot(2,2,2,projection=display.grid_projection)
        display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
                             colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
                             min_lon=getLocation(lat, lon, 270, r)[1],
                             max_lon=getLocation(lat, lon, 90, r)[1],
                             min_lat=getLocation(lat, lon, 180, r)[0],
                             max_lat=getLocation(lat, lon, 0, r)[0],
                             embellish=False)
        ax2.plot(lon,lat,'k*',ms=50,transform=ccrs.PlateCarree())
        ax2.plot(-104.4056,45.0223,'b*',ms=50,transform=ccrs.PlateCarree())
        #ax2.plot(-103.7701,44.59018,'b*',ms=50,transform=ccrs.PlateCarree())
        ax2.set_title(title)

        plt.suptitle(thefile.split('\\')[-1].split('.')[0][3:])#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
        fname = thefile.split('\\')[-1].split('.')[0][3:]
        fig.savefig(f'/Users/julia/Desktop/{fname}nondealise.png',dpi=300)
        plt.close('all')
        fig.clf()
        gc.collect()

In [1]:
# Julia's code testing

for thefile in vad_files[0:1]:

    radar = pyart.io.read(thefile)
    print(thefile,radar.fixed_angle['data'])

    df = pd.read_csv('C:\\Users\\julia\\OneDrive\\Desktop\\VADS Data Morton\\GPS_Ka1_20220612.txt', dtype=str)
    
    
    df['ddmmyy']+df['hhmmss[UTC]']
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amin(np.unique(radar.time['data']))
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.amax(np.unique(radar.time['data']))
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})

NameError: name 'vad_files' is not defined